In [2]:
import numpy as np
from qore import Mine, ASP, QAOA, VQE
from qore.algorithms.pseudoflow import Pseudoflow
from qore.utils import measure_operator, get_bitstring_probabilities

In [18]:
penalty = 10.0
qmine = Mine(np.array([[-2., 3., 1.], [float('inf'), 5., float('inf')]]))
qmine.plot_mine()

+---+--------+-------+-------+
|   |   0    |   1   |   2   |
+---+--------+-------+-------+
| 0 | -2.000 | 3.000 | 1.000 |
| 1 |  inf   | 5.000 |  inf  |
+---+--------+-------+-------+


In [19]:
graph, source, sink = qmine.gen_pseudoflow_graph()

In [20]:
pf = Pseudoflow(graph, source, sink)

In [21]:
pf.run()

{'ground_state': '1111'}

In [22]:
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit import Aer

In [23]:
aqua_globals.random_seed = 1953
backend = Aer.get_backend('statevector_simulator')
instance = QuantumInstance(backend=backend)

In [24]:
def analysis(circuit):
    print("Average Profit:                          ", measure_operator(qmine.Hp, circuit, instance))
    print("Average Number of Smoothness Violations: ", measure_operator(qmine.Hs, circuit, instance) / penalty)
    x = get_bitstring_probabilities(circuit, instance, shots=1024)
    bitstr, prob = max(x.items(), key=lambda item: item[1])
    print(f"The most probable configuration and the corresponding probability: {bitstr, prob}")
    qmine.plot_mine_state(bitstr)
    print("Current profit:               ", qmine.get_profit(bitstr))
    print("Current number of violations: ", qmine.get_violation(bitstr))
    print(x)
    print("------------------------------------------------------------")

In [27]:
#from qiskit.aqua.algorithms import QAOA, VQE
from qiskit.aqua.components.optimizers import COBYLA

p = 6
qaoa = QAOA(qmine.gen_Hamiltonian(penalty=penalty), optimizer=COBYLA(), quantum_instance=instance, p=p)
qaoa.run()
circuit = qaoa.get_optimal_circuit()

analysis(circuit)
# print(circuit.draw())
# print(circuit.decompose().draw())

Average Profit:                           6.280709518980992
Average Number of Smoothness Violations:  0.003650767433431429
The most probable configuration and the corresponding probability: ('1111', 0.8642578125)
+---+---+---+---+
|   | 0 | 1 | 2 |
+---+---+---+---+
| 0 | 1 | 1 | 1 |
| 1 | x | 1 | x |
+---+---+---+---+
Current profit:                7.0
Current number of violations:  0
{'0000': 0.001953125, '0001': 0.00390625, '0010': 0.0009765625, '0011': 0.0361328125, '0100': 0.0048828125, '0101': 0.0302734375, '0110': 0.017578125, '0111': 0.015625, '1000': 0.001953125, '1001': 0.0048828125, '1010': 0.0048828125, '1011': 0.0029296875, '1100': 0.00390625, '1110': 0.005859375, '1111': 0.8642578125}
------------------------------------------------------------


In [28]:
p = 4

QAOA(qmine.gen_Hamiltonian(penalty=penalty), optimizer=COBYLA(), quantum_instance=instance, p=p).run()

qaoa = QAOA(qmine.gen_Hamiltonian(penalty=penalty), optimizer=COBYLA(), quantum_instance=instance, p=p)
qaoa.run()
circuit = qaoa.get_optimal_circuit()

analysis(circuit)

Average Profit:                           3.090726295778179
Average Number of Smoothness Violations:  0.0026760071821298357
The most probable configuration and the corresponding probability: ('0010', 0.484375)
+---+---+---+---+
|   | 0 | 1 | 2 |
+---+---+---+---+
| 0 | 0 | 1 | 0 |
| 1 | x | 0 | x |
+---+---+---+---+
Current profit:                3.0
Current number of violations:  0
{'0000': 0.0771484375, '0010': 0.484375, '0011': 0.00390625, '0100': 0.060546875, '0110': 0.3603515625, '1000': 0.001953125, '1001': 0.0009765625, '1011': 0.0048828125, '1100': 0.001953125, '1101': 0.0009765625, '1110': 0.0029296875}
------------------------------------------------------------
